<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run All ZSMMT

Goal : train models on ZSMMT

Data : All ZSMMT

Models : SelfAttnLNP and ConvLNP and ConvCNP 

Loss : NLLLloss

Runs : 1

In [1]:
import os

os.chdir("..")

In [2]:
import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)


In [6]:
def load_reeval(name, n_test, bs=16, is_iw=False, loss=None):
    name = f"{name}_eval"
        
    args = get_exp_args(name, is_load=False, is_reeval=True)
    if loss is not None:
        args = [arg for arg in args if (arg.loss == loss)]
        
    for arg in args:
        arg.n_z_samples_test = n_test
        arg.batch_size=bs
        if is_iw:
            arg.is_iw_eval=True
    return args

In [17]:
args=[]
args.extend(load_reeval("exp_final3_allmm", 128, bs=4, is_iw=True, loss="ElboLNPF"))
args.extend(load_reeval("exp_final3_allmm", 128, bs=4, is_iw=False, loss="NllLNPF"))

In [18]:
len(args)

20

In [19]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24,  
                           cpus_per_task=10, 
                           mem='32GB',
                          partition="dev",
                           comment="neurips",
                           constraint="volta32gb"
                          )

In [20]:
jobs = executor.map_array(Run(), args)

In [21]:
############################################################

In [23]:
jobs

[SlurmJob<job_id=27235832_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_5, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_6, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_7, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_8, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_9, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_10, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_11, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_12, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_13, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_14, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_15, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27235832_16, task

In [21]:
for job in jobs:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-05-31 12:25:32,796) - Starting with JobEnvironment(job_id=27079441_0, hostname=learnfair0713, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-05-31 12:25:32,796) - Loading pickle: /private/home/yannd/projects/NPF/logs/27079441_0/27079441_0_submitted.pkl

--- Training zsmms/ConvNPFL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp        dur
-------  ------------  ------------  ----  ---------
      1     -521.5045     -891.3960     +  1981.3041
      2     -993.3695    -1098.7431     +  1977.8437
      3    -1069.7594    -1201.0772     +  1979.1721
      4    -1130.7543    -1165.4901        1977.7933
      5    -1224.4532    -1221.2071     +  1980.7182
      6    -1235.3471    -1325.3845     +  1977.8942
      7    -1250.8356    -1205.5548        1979.9167
      8    -1235.9238    -1295.1569        1980.1106
      9    -1276.4519    -1322.1920        1977.3992
     10    -1292.3487    -1411.2056     +  1

In [10]:
for j in jobs:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...

--------------------------------
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
INFO:utils.data.imgs:Resizing ZeroShotMultiMNIST ...
submitit ERROR (2020-05-29 11:31:58,066) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/priv

In [16]:
for j in jobs:
    j.cancel()